In [4]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
# import xgboost as xgb
# import lightgbm as lgb
# import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
# from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

### step9
* self train take others (fname,softmax) CSV

In [33]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test

folder = 'data/selftrain/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/selftrain/valid_acc.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
    df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
        train_X += df.values*ratio 
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))

(15163, 41)
1.0000000035901515


In [34]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test['label'] = 'none'
# fname_test
fname_un = pd.read_csv('data/train_label.csv')
fname_un = fname_un[fname_un.manually_verified==0]
fname_un = fname_un.drop(['manually_verified'],axis=1)
fname = fname_un.append(fname_test)
fname.sort_values('fname',inplace=True)
fname.reset_index(drop=True,inplace=True)
fname['label_verified'] = "none"
fname['verified_confidence']=0.0
print(len(fname))
for i,r in fname.iterrows():
    fname.iloc[i,2] = np.argmax(train_X[i])
    fname.iloc[i,3] = max(train_X[i])

df = fname
df

15163


,fname,label,label_verified,verified_confidence
0,00044347.wav,Hi-hat,0,0.508643
1,00063640.wav,none,30,0.524175
2,0013a1db.wav,none,16,0.846859
3,002bb878.wav,none,21,0.457352
4,002d256b.wav,Trumpet,19,0.126812
5,002d392d.wav,none,21,0.915956
6,00326aa9.wav,none,15,0.964103
7,0038a046.wav,none,21,0.590792
8,003995fa.wav,none,23,0.328664
9,003b91e8.wav,Cello,4,0.838155


In [35]:
dicts_ch = pickle.load(open('data/map_reverse.pkl','rb'))
df['label_verified'] = df['label_verified'].map(dicts_ch)
threshold_upper = df['verified_confidence'].mean() + df['verified_confidence'].std()
threshold_lower = min([df['verified_confidence'].mean() , 0.7])
print("threshold upper , lower:",threshold_upper , threshold_lower)
print("mean , std:",df['verified_confidence'].mean() , df['verified_confidence'].std())
df['take']=0

for i,r in df.iterrows():
    if r['verified_confidence']>threshold_upper:
        df.iloc[i,4]=1
    elif ((r['label'] == r['label_verified'])and(r['verified_confidence']>threshold_lower)):
        df.iloc[i,4]=1
    else:
        df.iloc[i,4]=0
df

threshold upper , lower: 0.962956929023524 0.6868034051035227
mean , std: 0.6868034051035227 0.27615352392000136


,fname,label,label_verified,verified_confidence,take
0,00044347.wav,Hi-hat,Hi-hat,0.508643,0
1,00063640.wav,none,Shatter,0.524175,0
2,0013a1db.wav,none,Flute,0.846859,0
3,002bb878.wav,none,Bass_drum,0.457352,0
4,002d256b.wav,Trumpet,Bark,0.126812,0
5,002d392d.wav,none,Bass_drum,0.915956,0
6,00326aa9.wav,none,Oboe,0.964103,1
7,0038a046.wav,none,Bass_drum,0.590792,0
8,003995fa.wav,none,Squeak,0.328664,0
9,003b91e8.wav,Cello,Cello,0.838155,1


In [36]:
df_ = df[df['take']==1]
df_ = df_.filter(['fname','label_verified'])
df_.to_csv('data/selftrain/Y_selftrain_ens_verified.csv',index=False) # 儲存train_Y
df_

,fname,label_verified
6,00326aa9.wav,Oboe
9,003b91e8.wav,Cello
13,0075d39c.wav,Keys_jangling
17,008afd93.wav,Saxophone
18,00a161c0.wav,Cello
20,00ad7068.wav,Laughter
28,00c9e799.wav,Oboe
29,00ccf065.wav,Bass_drum
30,00ce569f.wav,Hi-hat
31,00d0ab77.wav,Clarinet


### predict weigted acc

In [43]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test

folder = 'data/selftrain/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/selftrain/valid_acc.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
    df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
        train_X += df.values*ratio 
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(15163, 41)
1.0000000035901515


In [44]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test['label'] = 'none'
# fname_test
fname_un = pd.read_csv('data/train_label.csv')
fname_un = fname_un[fname_un.manually_verified==0]
fname_un = fname_un.drop(['manually_verified'],axis=1)
fname = fname_un.append(fname_test)
fname.sort_values('fname',inplace=True)
fname.reset_index(drop=True,inplace=True)
fname['label_verified'] = "none"
fname['verified_confidence']=0.0
print(len(fname))
for i,r in fname.iterrows():
    top3 = train_X[i].argsort()[-3:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)
    fname.iloc[i,2] = s#np.argmax(train_X[i])
    fname.iloc[i,3] = max(train_X[i])

df = fname
df

15163


,fname,label,label_verified,verified_confidence
0,00044347.wav,Hi-hat,Hi-hat Violin_or_fiddle Trumpet,0.508643
1,00063640.wav,none,Shatter Keys_jangling Scissors,0.524175
2,0013a1db.wav,none,Flute Meow Trumpet,0.846859
3,002bb878.wav,none,Bass_drum Knock Gunshot_or_gunfire,0.457352
4,002d256b.wav,Trumpet,Bark Tambourine Writing,0.126812
5,002d392d.wav,none,Bass_drum Knock Double_bass,0.915956
6,00326aa9.wav,none,Oboe Bass_drum Clarinet,0.964103
7,0038a046.wav,none,Bass_drum Double_bass Electric_piano,0.590792
8,003995fa.wav,none,Squeak Telephone Clarinet,0.328664
9,003b91e8.wav,Cello,Cello Acoustic_guitar Chime,0.838155


In [45]:
df_fin = df[df.label=='none']
df_fin = df_fin.filter(['fname','label_verified'])
df_fin.columns = ['fname','label']
df_fin.to_csv('result/cotrain_ens_oriACC.csv',index=False)
df_fin

,fname,label
1,00063640.wav,Shatter Keys_jangling Scissors
2,0013a1db.wav,Flute Meow Trumpet
3,002bb878.wav,Bass_drum Knock Gunshot_or_gunfire
5,002d392d.wav,Bass_drum Knock Double_bass
6,00326aa9.wav,Oboe Bass_drum Clarinet
7,0038a046.wav,Bass_drum Double_bass Electric_piano
8,003995fa.wav,Squeak Telephone Clarinet
11,005ae625.wav,Acoustic_guitar Gong Cello
14,007759c4.wav,Clarinet Saxophone Telephone
17,008afd93.wav,Saxophone Violin_or_fiddle Trumpet
